In [1]:
import sys
sys.path.append('../')
from model import NextWordPredictor, train, test

In [2]:
import pandas as pd
import torch
import torchtext.functional as F

In [3]:
train_d = pd.read_csv('../data/stack_overflow/train.csv')
test_d = pd.read_csv('../data/stack_overflow/test.csv')
val_d = pd.read_csv('../data/stack_overflow/val.csv')

In [4]:
train_sentence = train_d['text'][0]
test_sentence = test_d['text'][0]
val_sentence = val_d['text'][0]

In [5]:
words1 = train_sentence.split()
words2 = test_sentence.split()

In [6]:
vocab = set(words1 + words2)

In [7]:
word_to_index = { word: i for i, word in enumerate(vocab) }
index_to_word = { i: word for word, i in word_to_index.items() }

In [8]:
vocab_size = len(vocab)
vocab_size

7843

In [9]:
indices1 = [word_to_index[word] for word in words1]
indices2 = [word_to_index[word] for word in words2]

In [10]:
train_data = torch.tensor(indices1 + indices2[:-1], dtype=torch.long).view(1, -1)  # Combine both sentences for training
train_targets = torch.tensor(indices1[1:] + indices2, dtype=torch.long).view(1, -1)  # Targets are next words

In [11]:
test_data = torch.tensor(indices2, dtype=torch.long).view(1, -1)  # Test sequence
test_targets = torch.tensor(indices2, dtype=torch.long).view(1, -1)  # Test targets

In [12]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_data, train_targets), batch_size=1)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_data, test_targets), batch_size=1)

In [13]:
# Model parameters
embedding_dim = 10
hidden_dim = 10
epochs = 100

In [14]:
import torch.nn as nn
import torch.optim as optim

In [15]:
# Initialize model, criterion, and optimizer
model = NextWordPredictor(vocab_size, embedding_dim, hidden_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [16]:
train(model, train_loader, criterion, optimizer, epochs)

/home/nadun/Documents/projects/flower/.venv/lib/python3.10/site-packages/torch/autograd/__init__.py:266: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 1/100, Loss: 8.989895820617676
Epoch 2/100, Loss: 8.96225357055664
Epoch 3/100, Loss: 8.932378768920898
Epoch 4/100, Loss: 8.897782325744629
Epoch 5/100, Loss: 8.85665225982666
Epoch 6/100, Loss: 8.807570457458496
Epoch 7/100, Loss: 8.749128341674805
Epoch 8/100, Loss: 8.680109024047852
Epoch 9/100, Loss: 8.59981632232666
Epoch 10/100, Loss: 8.508410453796387
Epoch 11/100, Loss: 8.407147407531738
Epoch 12/100, Loss: 8.298277854919434
Epoch 13/100, Loss: 8.184587478637695
Epoch 14/100, Loss: 8.068978309631348
Epoch 15/100, Loss: 7.954272270202637
Epoch 16/100, Loss: 7.842989921569824
Epoch 17/100, Loss: 7.737094402313232
Epoch 18/100, Loss: 7.63785982131958
Epoch 19/100, Loss: 7.5459303855896
Epoch 20/100, Loss: 7.461393356323242
Epoch 21/100, Loss: 7.383922100067139
Epoch 22/100, Loss: 7.312985420227051
Epoch 23/100, Loss: 7.248074531555176
Epoch 24/100, Loss: 7.188785076141357
Epoch 25/100, Loss: 7.134804725646973
Epoch 26/100, Loss: 7.085860729217529
Epoch 27/100, Loss: 7.04166

In [17]:
test(model, test_loader, criterion)

Test Loss: 6.694679260253906
